In [1]:
import os
import json
import time
from tqdm import tqdm
import torch
from torch.profiler import profile, record_function, ProfilerActivity
import pandas as pd
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch.quantization
import torch.ao.quantization as tq
import gc

import jiwer
from jiwer import (
    Compose,
    ToLowerCase,
    RemoveMultipleSpaces,
    Strip,
)

# Final experiment
В этом ноутбуке проверяется гипотеза, что пропруненная ранее модель с базовой квантизацией и компиляцией даёт лучшие результаты. Прогон ноутбука произведён на тестовом сервере прунинга, поэтому результаты могут незначительно отличаться от метрик бэйзлайна.

Все значения профилировщика были сняты в прогоне на чистовую вне изолированной среды, поэтому значения могут отличаться, но статистически сводятся к итоговым метрикам.

In [2]:
def print_size_of_model(model):
    """ Prints the real size of the model """
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [3]:
def asr_metrics(hypothesis: str, reference: str):
    tr = Compose([ToLowerCase(), RemoveMultipleSpaces(), Strip()])

    ref_tr = tr(reference)
    hyp_tr = tr(hypothesis)

    out = jiwer.process_words(ref_tr, hyp_tr)
    wer = out.wer
    # S, D, I = out.substitutions, out.deletions, out.insertions

    cer = jiwer.cer(ref_tr, hyp_tr) # ?????

    return {
        "wer": wer,
        "cer": cer,
    }

In [4]:
def profile_sample(sample_idx=0, trace_path="whisper_perfetto_large-v3.json", sort_by="cpu_time_total", model=None):
    example = dataset[sample_idx]
    audio_array = example["audio"]["array"]
    sampling_rate = example["audio"]["sampling_rate"]

    inputs = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_features

    with profile(
        activities=[ProfilerActivity.CPU],
        record_shapes=True,
        profile_memory=True,
        with_stack=False,
    ) as prof:
        with record_function("whisper.generate"):
            predicted_ids = model.generate(inputs, forced_decoder_ids=forced_decoder_ids)

    prof.export_chrome_trace(trace_path)
    print(f"Perfetto trace saved to {trace_path}")
    print(prof.key_averages().table(
        sort_by=sort_by,
        row_limit=10
    ))
    return processor.decode(predicted_ids[0])

In [5]:
dataset = load_dataset("bond005/sberdevices_golos_10h_crowd", split="validation") #, split="test")
# dataset = dataset.select(range(100))

# --TO_DO - загрузить базовую модель здесь

In [6]:
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")
print_size_of_model(model)

Size (MB): 6174.372281


In [7]:
# Посмотрим на архитектуру модели
print(model)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias=Tr

In [45]:
def run_model(verbose=False, model=None, dataset=None):
    results = []
    i = 0
    for audio in tqdm(dataset):
        audio_array = audio["audio"]["array"]
        sampling_rate = audio["audio"]["sampling_rate"]
        reference = audio["transcription"]
    
        start_time = time.time()
        input_features = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_features 
        predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)[0] #Уточнить в зависимости от выбранной модели
        hypothesis = processor.decode(predicted_ids)
        run_time = time.time() - start_time
        metrics = asr_metrics(hypothesis, reference)
        metrics["run_time_sec"] = run_time
        if verbose:
            if i % 50 == 0:
                print("referenct:")
                print(reference)
                print("hypothesis:")
                print(hypothesis)
            i += 1
        results.append(metrics)

    df_results = pd.DataFrame(results)
    
    summary = {
        "total_samples": len(df_results),
        "avg_wer": df_results["wer"].mean(),
        "avg_cer": df_results["cer"].mean(),
        "avg_time_per_audio": df_results["run_time_sec"].mean(),
        "total_time": df_results["run_time_sec"].sum(),
    }
    
    print("large-v3")
    print(json.dumps(summary, ensure_ascii=True, indent=2))
    return summary

# CPU
Профилировка базовой модели

In [ ]:
_ = profile_sample(116, trace_path="whisper_perfetto_large-v3_base.json", model=model)

In [ ]:
del model, _
gc.collect()

# -- TO_DO Грузим запруненную модель

Профилировка запруненной модели

In [ ]:
pruned_model = WhisperForConditionalGeneration.from_pretrained("pruned_081")

In [ ]:
_ = profile_sample(116, trace_path="whisper_perfetto_large-v3_pruned_base.json", model=pruned_model)

In [15]:
dataset_small = dataset.select(range(50))
_ = run_model(model=model, dataset=dataset_small)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [06:50<00:00,  8.21s/it]

large-v3
{
  "total_samples": 50,
  "avg_wer": 0.3984776334776335,
  "avg_cer": 0.14465731826243972,
  "avg_time_per_audio": 8.201239647865295,
  "total_time": 410.06198239326477
}


In [17]:
del dataset_small, _
gc.collect()

1302245

# -- TO_DO Вывод

# PTQ Dynamic
Простейшая восьмибитная квантизация в одну строчку.

In [40]:
print_size_of_model(pruned_model)

Size (MB): 6174.372281


In [ ]:
modules_to_quantize = {torch.nn.Linear}
qmodel = tq.quantize_dynamic(
    pruned_model, 
    modules_to_quantize, 
    dtype=torch.qint8
)
print_size_of_model(qmodel)


In [ ]:
_ = profile_sample(116, trace_path="whisper_perfetto_large-v3_quanted_pruned.json", model=qmodel)

# -- TODO Вывод
Мы получили значительное ускорение инференса. Попробуем снять замеры качества с учётом torch.compile

In [43]:
qmodel = torch.compile(qmodel)
_ = profile_sample(116, trace_path="whisper_perfetto_large-v3_quanted_pruned_compiled.json", model=qmodel)

Perfetto trace saved to whisper_perfetto_large-v3_quanted_compiled.json
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-----------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                     whisper.generate         8.22%     634.960ms       100.00%        7.726s        7.726s         112 B      -6.85 GB             1  
                            quantized::linear_dynamic        48.54%        3.750s        49.43%        3.819s     929.023us       2.55 GB      -2.55 GB          4111  
                   aten::scaled_dot_product_attention         0.10%       7.414ms       

In [46]:
summary = run_model(verbose=True, model=qmodel, dataset=dataset)

  0%|▎                                                                                                                                                                                                            | 1/793 [00:06<1:27:06,  6.60s/it]

referenct:
можешь включить сериал теория большого взрыва
hypothesis:
 Можешь включить сериал «Теория большого взрыва»?


  6%|█████████████                                                                                                                                                                                               | 51/793 [04:59<1:16:07,  6.16s/it]

referenct:
покажи на смотрешке канал бридж тв
hypothesis:
 Покажи на сматрёшке канал Бридж ТВ.


 13%|█████████████████████████▊                                                                                                                                                                                 | 101/793 [09:55<1:03:30,  5.51s/it]

...

referenct:
вячеслав владимирович месяцев
hypothesis:
 Вячеслав Владимирович Месяцев


 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏          | 751/793 [1:12:49<03:59,  5.71s/it]

referenct:
футбольный матч тоттенхэм лестер
hypothesis:
 Футбольный матч Тоттенхэм-Лестер


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 793/793 [1:16:50<00:00,  5.81s/it]

large-v3
{
  "total_samples": 793,
  "avg_wer": 0.4739530218975364,
  "avg_cer": 0.16637995920858634,
  "avg_time_per_audio": 5.805704870861385,
  "total_time": 4603.923962593079
}


In [47]:
with open("whisper_metric.json", "r", encoding="utf-8") as f:
    data = json.load(f)

data["large-v3_cpu_quanted_pruned"] = summary

with open("whisper_metric.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=True, indent=2)

# --TO_DO Выведи только исходную метрику для CPU/GPU, large-v3_cpu_quanted, large-v3_cuda_global_magnitude_pruning_0.81 и large-v3_cpu_quanted_pruned

In [48]:
pd.DataFrame(data)

,tiny,small,iarge-v3,large-v3_cuda,large-v3_cpu_quanted,large-v3_cuda_quanted,large-v3_cuda_autocast_compile
total_samples,793.000000,793.000000,793.000000,793.000000,793.000000,793.000000,793.000000
avg_wer,1.049771,0.523011,0.440303,0.440303,0.473953,0.447231,0.442587
avg_cer,0.486142,0.207796,0.158293,0.158293,0.166380,0.158552,0.158944
avg_time_per_audio,0.168900,0.682818,7.948655,0.838627,5.805705,2.843514,0.997778
total_time,133.937801,541.474906,6303.283459,665.031494,4603.923963,2254.906898,791.237885


In [50]:
# Сохраняем state_dict квантованной модели
torch.save(qmodel.state_dict(), "./whisper-large-v3-quantized-dynamic-pruned.pth")

# Также сохраните конфигурацию отдельно (она не меняется)
qmodel.config.save_pretrained("./whisper-large-v3-quantized-dynamic-pruned")

In [52]:
del qmodel, _
gc.collect()

NameError: name 'qmodel' is not defined